# Ant Colony Optimisation Example

## 1. Initlialisation

#### Get distance matrix

In [1]:
# Create distance matrix (excluding distance from each point to itself)
import random
import numpy as np
import xml.etree.ElementTree as ET
from ipyparallel.util import interactive


burma_tree = ET.parse('burma14.xml')
burma_root = burma_tree.getroot()
brazil_tree = ET.parse('brazil58.xml')
brazil_root = brazil_tree.getroot()

d = []
edge_num = 14
country = "burma" #brazil, burma

if country == "brazil":
    edge_num = 58
    
    for vertex in brazil_root.iter('vertex'):
        temp = [0]*edge_num
        for edge in vertex.iter('edge'):
            temp[int(edge.text)] = (float(edge.get('cost')))
        d.append(temp)
else:
    for vertex in burma_root.iter('vertex'):
        temp = [0]*edge_num
        for edge in vertex.iter('edge'):
            temp[int(edge.text)] = (float(edge.get('cost')))
        d.append(temp)

    
d = np.array(d)

## 2. Creating base methods

#### Node Traversing Methods

In [2]:
def remove_current_city(city, H):
    """
    Removes current node from hueristic matrix

    :param city: the current node
    :param H: hueristic matrix
    :return: new heuristic matrix
    """ 
    
    H = [[round(1/d[i][j],4) if j != city and H[i][j] != 0 else 0 for j in range(edge_num)] for i in range(edge_num)]
    
#     for i in range(edge_num):
#         for j in range(edge_num):
#             if j != city and H[i][j] != 0:
#                 H[i][j] = round(1/d[i][j],4)
#             else:
#                 H[i][j] = 0
    return H




def find_next_city(P, city):
    """
    Determines the next node for an ant to go to

    :param P: probability matrix
    :param city: current node
    :return: next node
    """ 
    rand = random.random()
    CP = 0

    for i in range(edge_num):
        CP += P[i]
        if CP >= rand:
            return i


#### Create and run ants methods

In [3]:
import copy

def create_probs(city, H, T, alpha, beta):
    """
    Create probability matrix using heuristic and pheromones

    :param city: the current node
    :param H: hueristic matrix
    :param T: pheromone matrix
    :param alpha: alpha hyper parameter
    :param beta: beta hyper parameter
    :return: probability matrix
    """ 
    N = [None]*edge_num
    P = [None]*edge_num
    den = 0

    # for each value in row of current city, plug number into formula x^alpha * y^beta
    for j in range(edge_num):
        N[j] = T[city][j] **alpha * H[city][j] **beta
        den += N[j]

    for j in range(edge_num):
        P[j] = N[j]/den
    
    return P

def create_ants(num, alpha, beta):
    """
    Create set of ants

    :param num: number of ants to create
    :param alpha: alpha hyper parameter
    :param beta: beta hyper parameter
    :return: list of ant objects
    """ 
    # List to hold ants
    Ants = np.array([])

    # Initial probabilities
    P = create_probs(0, copy.deepcopy(H), copy.deepcopy(T), alpha, beta)

    # Create ants
    for i in range(num):
        Ants = np.append(Ants, {
            "Name": (i+1),
            "H": copy.deepcopy(H),
            "T": copy.deepcopy(T),
            "P": copy.deepcopy(P),
            "Length": 0,
            "Path" : [0]
        })
    return Ants
        
def run_ants(start, Ants, alpha, beta):
    """
    Run the ants over the nodes storing a path and length for each

    :param start: starting node
    :param Ants: list of ant objects
    :param alpha: alpha hyper parameter
    :param beta: beta hyper parameter
    :return: void
    """ 
        
    # For each ant
    for ant in Ants:

        # Start city
        current_city = start

        # Go to every node
        for i in range(edge_num - 1):
            # Remove this city from heuristics
            ant["H"] = remove_current_city(current_city, ant["H"])

            # Create probabilities
            ant["P"] = create_probs(current_city, ant["H"], ant["T"], alpha, beta)

            # Get next city to go to
            temp = current_city
            current_city = find_next_city(ant["P"], current_city)

            # Update length
            ant["Length"] += d[temp][current_city]

            # Update path
            ant["Path"].append(current_city)
            
        # Add the length to get back to starting node
        ant["Length"] += d[current_city][start]

#### Evapourate and deposit methods

In [4]:
# Evapourate Pheromones
def evapourate(e):
    """
    Evapourate pheromones

    :param e: evapouration rate
    :return: void
    """ 
            
    return np.multiply(T, (1-e))

def pheromone_update(Ants, Q):
    """
    Updates pheromones for acs approach

    :param Ants: list of ant objects
    :param Q: hyper parameter to scale pheromones
    :return: void
    """
    
    for ant in Ants:
        # Determine the amount to deposit
        delta = Q/ant["Length"]
        
        # Deposit on each path taken
        x = [[item] for item in ant["Path"][1::]]
        y = [[item] for item in ant["Path"][:-1]]
        indices = np.hstack([y, x])

        np.add.at(T, (indices[:, 0], indices[:, 1]), delta)

    
#     # Loop through every ant
#     for ant in Ants:
#         # Determine the amount to deposit
#         delta = Q/ant["Length"]

#         # Deposit on each path taken
#         for i in range(edge_num - 1):
#             T[ant["Path"][i]][ant["Path"][i+1]] += delta
            
    return T
            
def pheromone_update_mmas(Ants, Q, global_best, upper_limit, lower_limit, mmas_type):
    """
    Updates pheromones for mmas approach

    :param Ants: list of ant objects
    :param Q: hyper parameter to scale pheromones
    :param global_best: global best path and length
    :param upper_limit: upper limit for pheromones
    :param lower_limit: lower limit for pheromones
    :param mmas_type: type of mmas (local or global)
    :return: void
    """ 
    
    # Global or Local mmas
    if mmas_type == "global":
        best_length = global_best
    elif mmas_type == "local":
        # Find local best
        best_length = find_best(Ants)["Length"]
           
    # Loop through every ant
    for ant in Ants:
        # If better than or equal to global/local best length then deposit
        if ant["Length"] <= best_length:
            # Determine the amount to deposit
            delta = Q/ant["Length"]

            # Limits for pheromone values
            if delta > upper_limit: delta = upper_limit
            elif delta < lower_limit: delta = lower_limit
                
            # Deposit on each path taken
            x = [[item] for item in ant["Path"][1::]]
            y = [[item] for item in ant["Path"][:-1]]
            indices = np.hstack([y, x])

            np.add.at(T, (indices[:, 0], indices[:, 1]), delta)
        
    return T
        
def pheromone_update_elitist(Ants, Q, global_best):
    """
    Updates pheromones for elitist approach

    :param Ants: list of ant objects
    :param Q: hyper parameter to scale pheromones
    :param global_best: global best path and length
    :return: void
    """ 
    
    # Find local best
    local_best = find_best(Ants)
    
    # If local is worse than global then deposit global
    if local_best["Length"] < global_best["Length"]:
        
        # Determine the amount to deposit
        delta = Q/global_best["Length"]
        
        # Deposit on each path taken
        for i in range(edge_num - 1):
            T[global_best["Path"][i]][global_best["Path"][i+1]] += delta
    
    # Loop through every ant
    for ant in Ants:
        # Determine the amount to deposit
        delta = Q/ant["Length"]

        # Deposit on each path taken
        x = [[item] for item in ant["Path"][1::]]
        y = [[item] for item in ant["Path"][:-1]]
        indices = np.hstack([y, x])

        np.add.at(T, (indices[:, 0], indices[:, 1]), delta)
    return T
            
def find_best(Ants):
    """
    Find the best solution

    :param Ants: list of ant objects
    :return: best solution (path and length)
    """ 
    
    # Find the smallest length
    smallest = Ants[0]["Length"]
    path = Ants[0]["Path"]
    for ant in Ants:
        if ant["Length"] < smallest:
            smallest = ant["Length"]
            path = ant["Path"]

    return {"Length": smallest, "Path": path}

## Running the optimisation - Parallel

#### Method to create and run ants

In [5]:
import ipyparallel as ipp

# Main loop
def create_and_run_ants(num_ants, starting_city, alpha, beta, Q, mmas, approach_type, e):
    """
    Create and run a set of ants

    :param num_ants: number of ants to create
    :param starting_city: starting node
    :param alpha: alpha hyper parameter
    :param beta: beta hyper parameter
    :param Q: hyper parameter to scale pheromones
    :param mmas: mmas properties
    :param approach_type: aco variation
    :param e: evapouration rate
    :return: list of ant objects
    """ 
    Ants = np.array([])
    # Create and run ants
    Ants = create_ants(num_ants, alpha, beta)
    run_ants(starting_city, Ants, alpha, beta)
        
    return Ants

#### Method to run ACO

In [6]:
# Define Global Pheromones (random by defaultbut overridden if needed)
T = np.array([[random.random() for j in range(edge_num)] for i in range(edge_num)])

# Initial Hueristic Matrix
H = np.array([[ round(1/d[i][j],4) if i != j else 0 for j in range(edge_num)] for i in range(edge_num)])

def run_ACO():
    """
    Run the ACO in parallel

    :return: global best solution
    """ 
    
    # Start n clusters and define variables in correct space
    with ipp.Cluster(n=10) as rc:
        view = rc[:]
        view.execute("import copy, random")
        view.execute("import numpy as np")
        view['H'] = H
        view['create_probs'] = create_probs
        view['create_ants'] = create_ants
        view['T'] = T
        view['remove_current_city'] = remove_current_city
        view['find_next_city'] = find_next_city
        view['edge_num'] = edge_num
        view['run_ants'] = run_ants
        view['d'] = d


        # Experimental parameters
        num_ants = 100
        num_clusters = 10
        Q = 1
        e = 0.7
        alpha = 1
        beta = 2
        approach_type = "mmas" # ("mmas", "elitist", "acs")
        mmas = {"upper": 0.6, "lower": 0.4, "type": "local"} # (type: "global", "local")

        # Base parameters
        starting_city = 0
        num_iterations = round(10_000 / (num_ants * num_clusters))
        global_best = 0
        Ants = []
        
        # If mmas then pheromones start on upper limit instead of random
        if approach_type == "mmas":
            view['T'] = [[mmas["upper"] for j in range(edge_num)] for i in range(edge_num)]

        # Loop for number of iterations
        for i in range(num_iterations):
            # Run async task to create and run ants
            asyncresult = view.apply_async(create_and_run_ants, num_ants, starting_city, 
                                           alpha, beta, Q, mmas, approach_type, e)
            asyncresult.wait_interactive()
            result = asyncresult.get()

            # Flatten sets of ants into one array
            Ants = [ant for cluster in result for ant in cluster]

            # Find local best
            local_best = find_best(Ants)

            # Set global best
            if i == 0 or (local_best["Length"] < global_best["Length"]):
                global_best = local_best

            # Check approach variation
            match approach_type:
                case "mmas":
                    # Use limited depositing
                    view['T'] = pheromone_update_mmas(Ants, Q, global_best["Length"], mmas["upper"], mmas["lower"], mmas["type"])
                case "elitist":
                    # Global best always deposits
                    view['T'] = pheromone_update_elitist(Ants, Q, global_best)
                case "acs":
                    # No limits on deposits
                    view['T'] = pheromone_update(Ants, Q)

            # Evapourate pheromones
            view['T'] = evapourate(e)

    # Return best solution
    return global_best

#### Run 5 ACOs

In [7]:
# Start 5 clusters to run 5 ACOs
with ipp.Cluster(n=5) as cl:
    viewcl = cl[:]
    viewcl.execute("import ipyparallel as ipp")
    viewcl.execute("import numpy as np")
    viewcl['H'] = H
    viewcl['create_probs'] = create_probs
    viewcl['create_ants'] = create_ants
    viewcl['T'] = T
    viewcl['remove_current_city'] = remove_current_city
    viewcl['find_next_city'] = find_next_city
    viewcl['edge_num'] = edge_num
    viewcl['run_ants'] = run_ants
    viewcl['d'] = d
    viewcl['create_and_run_ants'] = create_and_run_ants
    viewcl['find_best'] = find_best
    viewcl['pheromone_update'] = pheromone_update
    viewcl['pheromone_update_mmas'] = pheromone_update_mmas
    viewcl['pheromone_update_elitist'] = pheromone_update_elitist
    viewcl['evapourate'] = evapourate
    asyncresult = viewcl.apply_async(run_ACO)
    asyncresult.wait_interactive()
    result = asyncresult.get()

# Print best solution for each ACO
for solution in result:
    print(f'Length: {solution["Length"]}, Path: {solution["Path"]}')

Starting 5 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]

run_ACO:   0%|          | 0/5 [00:00<?, ?tasks/s]

Stopping engine(s): 1702308041
engine set stopped 1702308041: {'engines': {'0': {'exit_code': 1, 'pid': 61744, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 58300, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 56288, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 41532, 'identifier': '3'}, '4': {'exit_code': 1, 'pid': 28220, 'identifier': '4'}}, 'exit_code': 1}
Stopping controller
Length: 3371.0, Path: [0, 7, 12, 6, 11, 5, 4, 3, 2, 13, 1, 9, 8, 10]
Length: 3381.0, Path: [0, 7, 10, 8, 9, 12, 6, 11, 5, 4, 3, 2, 13, 1]
Length: 3381.0, Path: [0, 7, 10, 8, 9, 12, 6, 11, 5, 4, 3, 2, 13, 1]
Length: 3381.0, Path: [0, 7, 10, 8, 9, 12, 6, 11, 5, 4, 3, 2, 13, 1]
Length: 3336.0, Path: [0, 7, 9, 8, 10, 12, 6, 11, 5, 4, 3, 2, 13, 1]
Controller stopped: {'exit_code': 1, 'pid': 48524, 'identifier': 'ipcontroller-1702308040-hsq7-64464'}


## Running the optimisation - Sequential

## Experiments